In [96]:
DOWNLOAD_LATEST = False
import os
if DOWNLOAD_LATEST:
    env_vars = open(".env.development").read()
    DB_CONNECTION_STRING = env_vars.split("DB_CONNECTION_STRING = ")[1].split("?authSource")[0]
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection users --out data/Curio/users.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection posts --out data/Curio/posts.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection groups --out data/Curio/groups.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection reactions --out data/Curio/reactions.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection echopostproposals --out data/Curio/echopostproposals.json")
    os.system(f"mongoexport --uri {DB_CONNECTION_STRING} --collection viewtimestamps --out data/Curio/viewtimestamps.json")

In [97]:
import jsonlines
import pandas as pd
from superdebug import debug
from collections import defaultdict
curio_data_path = "data/Curio/"
users = jsonlines.Reader(open(curio_data_path + "users.json"))
posts = jsonlines.Reader(open(curio_data_path + "posts.json"))
groups = jsonlines.Reader(open(curio_data_path + "groups.json"))
reactions = jsonlines.Reader(open(curio_data_path + "reactions.json"))
echopostproposals = jsonlines.Reader(open(curio_data_path + "echopostproposals.json"))
viewtimestamps = jsonlines.Reader(open(curio_data_path + "viewtimestamps.json"))


##### Record necessary information: `group_id_name_map, group_id_members_id_map, user_id_name_map, user_id_group_id_viewtime_map`

In [98]:
user_id_name_map = {}
for user in users:
    user_id = user['_id']["$oid"]
    user_name = user["username"]
    user_id_name_map[user_id] = user_name
    
group_id_members_id_map = {}
group_id_members_name_map = {}
group_name_member_num_map = {}
group_id_name_map = {}
for group in groups:
    group_id = group['_id']["$oid"]
    group_name = group["name"].replace(" ", "|")
    group_id_name_map[group_id] = group_name
    members = [_["$oid"] for _ in group['members']]
    group_id_members_id_map[group_id] = members
    group_id_members_name_map[group_name] = [user_id_name_map[user_id] for user_id in members if user_id in user_id_name_map]
    if "TEST" not in group_name:
        group_name_member_num_map[group_name] = len(members)
debug(group_member_num = group_name_member_num_map)
print("group_id_members_name_map:", group_id_members_name_map)
    
user_id_group_id_viewtime_map = defaultdict(dict)
for viewtimestamp in viewtimestamps:
    if viewtimestamp["user"] is None: continue
    user_id = viewtimestamp["user"]["$oid"]
    if "group" not in viewtimestamp or viewtimestamp["group"] == None:
        group_ids = []
        for group_id in group_id_members_id_map:
            if user_id in group_id_members_id_map[group_id]:
                group_ids.append(group_id)
    else:
        group_ids = [viewtimestamp["group"]["$oid"]]
    dt_obj = pd.to_datetime(viewtimestamp["timestamp"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    for group_id in group_ids:
        if group_id not in user_id_group_id_viewtime_map[user_id] or timestamp > user_id_group_id_viewtime_map[user_id][group_id]:
            user_id_group_id_viewtime_map[user_id][group_id] = timestamp
    

------------------ 2022-11-06 02:31:08 ------------------
DEBUG: 1 vars: ['group_member_num'], at <ipython-input-98-5d1bfe551d68>:20 <module>
0 / 56.  group_member_num dict {...} with 14 keys ['Trivia', 'LOLCHI', 'Team|Mignon', 'The|Positive|Corner', 'Random', 'Out|of|Context|Quotes', 'Team|PhThree', 'Team|Llama', 'Food', 'Stanford|HCI', 'STANFORD', 'Campus|Events', 'Stanford|Wordle', 'Where|Am|I']
    Trivia num val: 18
    LOLCHI num val: 23
    Team|Mignon num val: 9
    The|Positive|Corner num val: 16
    Random num val: 17
    Out|of|Context|Quotes num val: 16
    Team|PhThree num val: 7
    Team|Llama num val: 7
    Food num val: 21
    Stanford|HCI num val: 22
    STANFORD num val: 4
    Campus|Events num val: 5
    Stanford|Wordle num val: 2
    Where|Am|I num val: 6
------------------ 2022-11-06 02:31:08 ------------------
group_id_members_name_map: {'Parent|(TEST)': ['parth', 'msb', 'parnoud', 'kjeong', 'msbtest4', 'Gayoung', 'jigar'], 'Trivia': ['wanrong', 'nicolejg', 'parno

##### Aggregate information about each post, and compose `posts_df`

In [99]:
import time
from collections import defaultdict, Counter
posts_info_map = defaultdict(dict)
group_name_post_num_map = defaultdict(int)
group_name_first_post_time_map = defaultdict(int)
group_name_active_post_member_map = defaultdict(Counter)
group_name_active_member_map = defaultdict(Counter)
active_member_day_thres = 200
for post in posts:
    group_id = post["group"]["$oid"]
    group_name = group_id_name_map[group_id]
    if "TEST" in group_name or "test" in group_name:
        continue
    
    group_name_post_num_map[group_name] += 1
    
    post_id = post["_id"]["$oid"]
    post_info = posts_info_map[post_id] # information about the post
    post_info["SUBMISSION_ID"] = post_id
    
    post_info["SUBREDDIT"] = group_name
    post_info["group_id"] = group_id
    title = post["contents"]
    post_info["TITLE"] = title
    
    author_id = post["author"]["$oid"]
    author = user_id_name_map[author_id]
    post_info["author_id"] = author_id
    post_info["AUTHOR"] = author
    post_info["NSFW"] = None
    post_info["LINK"] = ""
    
    dt_obj = pd.to_datetime(post["updatedAt"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    post_info["timestamp"] = timestamp
    if (group_name not in group_name_first_post_time_map) or timestamp < group_name_first_post_time_map[group_name]:
        group_name_first_post_time_map[group_name] = timestamp
    
    if abs(time.time() - timestamp) <= active_member_day_thres * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name][author_id] += 1
        group_name_active_post_member_map[group_name][author_id] += 1
        
    
    if "#_COMMENTS" not in post_info:
        post_info["#_COMMENTS"] = 0
        
    if "threadParent" in post:
        threadParent_id = post["threadParent"]["$oid"]
        if "#_COMMENTS" not in posts_info_map[threadParent_id]:
            posts_info_map[threadParent_id]["#_COMMENTS"] = 1
        else:
            posts_info_map[threadParent_id]["#_COMMENTS"] += 1

debug(group_active_post_member_num = str({name: len(members) for name, members in group_name_active_post_member_map.items()}))
debug(group_active_post_member_num_2 = str({name: len([member for member in members if members[member] >=2]) for name, members in group_name_active_post_member_map.items()}))

------------------ 2022-11-06 02:31:09 ------------------
DEBUG: 1 vars: ['group_active_post_member_num'], at <ipython-input-99-b691f67c88c2>:54 <module>
0 / 57.  group_active_post_member_num str len 229: {'LOLCHI': 9, 'Random': 7, 'Team|Mignon': 4, 'Team|Llama': 5, 'The|Positive|Corner': 9, 'Food': 6, 'Team|PhThree': 3, 'Out|of|Context|Quotes': 8, 'Trivia': 9, 'Stanford|HCI': 1, 'Campus|Events': 2, 'Where|Am|I': 4, 'STANFORD': 1}
------------------ 2022-11-06 02:31:09 ------------------
------------------ 2022-11-06 02:31:09 ------------------
DEBUG: 1 vars: ['group_active_post_member_num_2'], at <ipython-input-99-b691f67c88c2>:55 <module>
0 / 58.  group_active_post_member_num_2 str len 229: {'LOLCHI': 6, 'Random': 5, 'Team|Mignon': 4, 'Team|Llama': 4, 'The|Positive|Corner': 8, 'Food': 6, 'Team|PhThree': 3, 'Out|of|Context|Quotes': 7, 'Trivia': 6, 'Stanford|HCI': 0, 'Campus|Events': 1, 'Where|Am|I': 3, 'STANFORD': 0}
------------------ 2022-11-06 02:31:09 ------------------


Analyze statistics

In [100]:
group_name_post_frequency_map = {group_name: (60 * 60 * 24 * group_name_post_num_map[group_name])/(time.time() - group_name_first_post_time_map[group_name]) for group_name in group_name_post_num_map}
print({group_name: f"{group_name_post_frequency_map[group_name]:.4f}" for group_name in group_name_post_frequency_map})
debug(group_posting_frequency_day = {group_name: f"{group_name_post_frequency_map[group_name]:.4f}" for group_name in group_name_post_frequency_map})

group_name_post_frequency_per_person_map = {group_name: group_name_post_frequency_map[group_name]/group_name_member_num_map[group_name] for group_name in group_name_post_frequency_map}
print({group_name: f"{group_name_post_frequency_per_person_map[group_name]:.4f}" for group_name in group_name_post_frequency_per_person_map})
debug(group_posting_frequency_per_person_day = {group_name: f"{group_name_post_frequency_per_person_map[group_name]:.4f}" for group_name in group_name_post_frequency_per_person_map})
posts_df = pd.DataFrame.from_records(list(posts_info_map.values()))

{'LOLCHI': '0.1628', 'Random': '0.2003', 'Team|Mignon': '0.8139', 'Team|Llama': '0.6374', 'The|Positive|Corner': '0.4851', 'Food': '0.2639', 'Team|PhThree': '0.1383', 'Out|of|Context|Quotes': '0.5130', 'Trivia': '0.3842', 'Stanford|HCI': '0.0394', 'Campus|Events': '0.6555', 'Where|Am|I': '0.9825', 'STANFORD': '0.0820'}
------------------ 2022-11-06 02:31:09 ------------------
DEBUG: 1 vars: ['group_posting_frequency_day'], at <ipython-input-100-43d8415bbca7>:3 <module>
0 / 59.  group_posting_frequency_day dict {...} with 13 keys ['LOLCHI', 'Random', 'Team|Mignon', 'Team|Llama', 'The|Positive|Corner', 'Food', 'Team|PhThree', 'Out|of|Context|Quotes', 'Trivia', 'Stanford|HCI', 'Campus|Events', 'Where|Am|I', 'STANFORD']
    LOLCHI str len 6: 0.1628
    Random str len 6: 0.2003
    Team|Mignon str len 6: 0.8139
    Team|Llama str len 6: 0.6374
    The|Positive|Corner str len 6: 0.4851
    Food str len 6: 0.2639
    Team|PhThree str len 6: 0.1383
    Out|of|Context|Quotes str len 6: 0.5130
 

##### Record users' votes on posts and compose `votes_df`

In [101]:
posts_users_votes = defaultdict(dict)
for post_id, post_info in posts_info_map.items():
    if "SUBMISSION_ID" not in post_info or "SUBREDDIT" not in post_info: continue
    if post_info["SUBREDDIT"] == "" or post_info["SUBREDDIT"] == None: continue
    assert post_id == post_info["SUBMISSION_ID"]
    post_timestamp = post_info["timestamp"]
    group_id = post_info["group_id"]
    author_id = post_info["author_id"]
    group_name = post_info["SUBREDDIT"]
    group_members = group_id_members_id_map[group_id]
    
    # fill in downvotes
    users_votes = posts_users_votes[post_id]
    for member_id in group_members:
        if member_id in user_id_name_map and (group_id not in user_id_group_id_viewtime_map[member_id] or post_timestamp < user_id_group_id_viewtime_map[member_id][group_id]): # TODO:
            users_votes[member_id] = {"SUBMISSION_ID": post_id,	"SUBREDDIT": "r/" + group_name, "CREATED_TIME": post_timestamp, "USERNAME": user_id_name_map[member_id], "VOTE": "downvote"}
    if author_id in users_votes:
        users_votes[author_id]["VOTE"] = "upvote" # author always upvotes
    else:
        users_votes[author_id] = {"SUBMISSION_ID": post_id,	"SUBREDDIT": "r/" + group_name, "CREATED_TIME": post_timestamp, "USERNAME": user_id_name_map[member_id], "VOTE": "upvote"}
        

In [102]:
for reaction in reactions:
    post_id = reaction["post"]["$oid"]
    if post_id not in posts_users_votes: continue
    author_id = reaction["author"]["$oid"]
    if author_id in posts_users_votes[post_id]:
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
    else:
        posts_users_votes[post_id][author_id] = list(posts_users_votes[post_id].values())[0]
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        posts_users_votes[post_id][author_id]["USERNAME"] = user_id_name_map[author_id]
        
    group_id = posts_info_map[post_id]["group_id"]
    group_name = group_id_name_map[group_id]
    dt_obj = pd.to_datetime(reaction["updatedAt"]["$date"], infer_datetime_format=True)
    timestamp = dt_obj.timestamp()
    if abs(time.time() - timestamp) <= active_member_day_thres * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name][author_id] += 1

for echopostproposal in echopostproposals:
    post_id = echopostproposal["echoParent"]["$oid"]
    if post_id not in posts_users_votes: continue
    author_id = echopostproposal["echoAuthor"]["$oid"]
    if author_id not in posts_users_votes[post_id]:
        posts_users_votes[post_id][author_id] = list(posts_users_votes[post_id].values())[0]
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        posts_users_votes[post_id][author_id]["USERNAME"] = user_id_name_map[author_id]
    else:
        posts_users_votes[post_id][author_id]["VOTE"] = "upvote"
        
    group_id = posts_info_map[post_id]["group_id"]
    group_name = group_id_name_map[group_id]
    
    post_timestamp = posts_info_map[post_id]["timestamp"]
    if abs(time.time() - post_timestamp) <= active_member_day_thres * 24 * 60 * 60 and author_id in group_id_members_id_map[group_id]:
        group_name_active_member_map[group_name][author_id] += 1

        


In [103]:
all_votes = []
group_name_upvote_num_map = defaultdict(int)
for post_id, users_votes in posts_users_votes.items():
    for user_id, vote in users_votes.items():
        all_votes.append(vote)
        if vote["VOTE"] == "upvote":
            group_name_upvote_num_map[vote["SUBREDDIT"][2:]] += 1
# group_name_upvote_frequency_map = {group_name: f"{(time.time() - group_name_first_post_time_map[group_name])/(60 * 60 * 24 * group_name_upvote_num_map[group_name]):.4f}" for group_name in group_name_upvote_num_map}
# debug(group_upvoteing_frequency_day = group_name_upvote_frequency_map)

group_name_upvote_frequency_map = {group_name: (60 * 60 * 24 * group_name_upvote_num_map[group_name])/(time.time() - group_name_first_post_time_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_frequency_map[group_name]:.4f}" for group_name in group_name_upvote_frequency_map})
debug(group_posting_frequency_day = {group_name: f"{group_name_upvote_frequency_map[group_name]:.4f}" for group_name in group_name_upvote_frequency_map})
debug(group_active_member_num = str({name: len(members) for name, members in group_name_active_member_map.items()}))
debug(group_active_member_num_2 = str({name: len([member for member in members if members[member] >=2]) for name, members in group_name_active_member_map.items()}))
votes_df = pd.DataFrame.from_records(all_votes)

{'LOLCHI': '0.4315', 'Random': '0.5100', 'Team|Mignon': '0.9785', 'Team|Llama': '0.7467', 'The|Positive|Corner': '1.1808', 'Food': '0.7037', 'Team|PhThree': '0.1778', 'Out|of|Context|Quotes': '1.5292', 'Trivia': '0.5580', 'Stanford|HCI': '0.1575', 'Campus|Events': '1.1471', 'Where|Am|I': '1.3100', 'STANFORD': '0.0820'}
------------------ 2022-11-06 02:31:10 ------------------
DEBUG: 1 vars: ['group_posting_frequency_day'], at <ipython-input-103-4aab69a28613>:13 <module>
0 / 61.  group_posting_frequency_day dict {...} with 13 keys ['LOLCHI', 'Random', 'Team|Mignon', 'Team|Llama', 'The|Positive|Corner', 'Food', 'Team|PhThree', 'Out|of|Context|Quotes', 'Trivia', 'Stanford|HCI', 'Campus|Events', 'Where|Am|I', 'STANFORD']
    LOLCHI str len 6: 0.4315
    Random str len 6: 0.5100
    Team|Mignon str len 6: 0.9785
    Team|Llama str len 6: 0.7467
    The|Positive|Corner str len 6: 1.1808
    Food str len 6: 0.7037
    Team|PhThree str len 6: 0.1778
    Out|of|Context|Quotes str len 6: 1.5292


In [104]:
group_name_upvote_rate_map = {group_name: (group_name_upvote_num_map[group_name])/(group_name_post_num_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_rate_map[group_name]:.4f}" for group_name in group_name_upvote_rate_map})
group_name_upvote_rate_per_person_map = {group_name: (group_name_upvote_num_map[group_name])/(group_name_post_num_map[group_name] * group_name_member_num_map[group_name]) for group_name in group_name_upvote_num_map}
print({group_name: f"{group_name_upvote_rate_per_person_map[group_name]:.4f}" for group_name in group_name_upvote_rate_per_person_map})

{'LOLCHI': '2.6500', 'Random': '2.5455', 'Team|Mignon': '1.2022', 'Team|Llama': '1.1714', 'The|Positive|Corner': '2.4340', 'Food': '2.6667', 'Team|PhThree': '1.2857', 'Out|of|Context|Quotes': '2.9808', 'Trivia': '1.4524', 'Stanford|HCI': '4.0000', 'Campus|Events': '1.7500', 'Where|Am|I': '1.3333', 'STANFORD': '1.0000'}
{'LOLCHI': '0.1152', 'Random': '0.1497', 'Team|Mignon': '0.1336', 'Team|Llama': '0.1673', 'The|Positive|Corner': '0.1521', 'Food': '0.1270', 'Team|PhThree': '0.1837', 'Out|of|Context|Quotes': '0.1863', 'Trivia': '0.0807', 'Stanford|HCI': '0.1818', 'Campus|Events': '0.3500', 'Where|Am|I': '0.2222', 'STANFORD': '0.2500'}


In [105]:
posts_df.to_csv(curio_data_path + "post_info.txt", sep = "\t", index = False)
votes_df.to_csv(curio_data_path + "all_votes.txt", sep = "\t", index = False)